GBDT

数据

|x|1|2|3|4|5|6|7|8|9|10
|--|--|--|--|--|--|--|--|--|--|--|
|y|4.50|4.75|4.91|5.34|5.80|7.05|7.90|8.23|8.70|9.00|

我们以 5.5 为分割点，将数据集一分为二，并将各自的平均值作为其预测值

![avatar](test.png)

In [3]:
import numpy as np
x = np.arange(1,11)
y = np.array([4.50,4.75,4.91,5.34,5.80,7.05,7.90,8.23,8.70,9.00])

In [5]:
sum(y[:5])/5

5.0600000000000005

In [6]:
sum(y[5:])/5

8.175999999999998

|x|1|2|3|4|5|6|7|8|9|10
|--|--|--|--|--|--|--|--|--|--|--|
|y|4.50|4.75|4.91|5.34|5.80|7.05|7.90|8.23|8.70|9.00|
|预测值|5.06|5.06|5.06|5.06|5.06|8.18|8.18|8.18|8.18|8.18|

与上节中以均方误差衡量预测效果不同，这里我们计算预测的残差

In [13]:
predict = np.array([5.06]*5+[8.18]*5)
diff = y - predict
diff

array([-0.56, -0.31, -0.15,  0.28,  0.74, -1.13, -0.28,  0.05,  0.52,
        0.82])

|x|1|2|3|4|5|6|7|8|9|10
|--|--|--|--|--|--|--|--|--|--|--|
|y|4.50|4.75|4.91|5.34|5.80|7.05|7.90|8.23|8.70|9.00|
|预测值|5.06|5.06|5.06|5.06|5.06|8.18|8.18|8.18|8.18|8.18|
|残差|-0.56| -0.31| -0.15|  0.28|  0.74| -1.13| -0.28|  0.05|  0.52| 0.82

GBDT 的下一步就是用一棵新的回归树来对上一步的残差进行预测，此时的数据集变成了：

|x|1|2|3|4|5|6|7|8|9|10
|--|--|--|--|--|--|--|--|--|--|--|
|残差|-0.56| -0.31| -0.15|  0.28|  0.74| -1.13| -0.28|  0.05|  0.52| 0.82


构建回归树的方法与此前完全相同：
首先选择 1.5 为划分点：

|x|1|2|3|4|5|6|7|8|9|10
|--|--|--|--|--|--|--|--|--|--|--|
|残差|-0.56| -0.31| -0.15|  0.28|  0.74| -1.13| -0.28|  0.05|  0.52| 0.82
|残差预测值|-0.56|0.06|0.06|0.06|0.06|0.06|0.06|0.06|0.06|0.06|

In [12]:
predict_2 = np.array([-0.56]+[0.06]*9)
predict_2

array([-0.56,  0.06,  0.06,  0.06,  0.06,  0.06,  0.06,  0.06,  0.06,
        0.06])

计算一下以 1.5 为划分点的 mse

In [15]:
sum(((diff-predict_2))**2)

3.0127999999999995

同样地，我们继续计算以 2.5, 3.5, ..., 9.5 为划分点时的预测 mse

|划分点|1.5|2.5|3.5|4.5|5.5|6.5|7.5|8.5|9.5|
|--|--|--|--|--|--|--|--|--|--|
|mse|3.01| ...|2.87|

经过计算我们发现以 3.5 为划分点时，预测 mse 最小

|x|1|2|3|4|5|6|7|8|9|10
|--|--|--|--|--|--|--|--|--|--|--|
|残差|-0.56| -0.31| -0.15|  0.28|  0.74| -1.13| -0.28|  0.05|  0.52| 0.82
|残差预测值|-0.34|-0.34|-0.34|0.14|0.14|0.14|0.14|0.14|0.14|0.14|

此时我们得到了第二颗树

![avatar](gbdt.png)

现在我们就得到了一个包含有两颗树的 GBDT 模型，它的预测值为第一颗树的预测值和第二颗树的预测值的和

In [28]:
cur_predict = predict + predict_2
cur_predict

array([4.72, 4.72, 4.72, 5.2 , 5.2 , 8.32, 8.32, 8.32, 8.32, 8.32])

|x|1|2|3|4|5|6|7|8|9|10
|--|--|--|--|--|--|--|--|--|--|--|
|y|4.50|4.75|4.91|5.34|5.80|7.05|7.90|8.23|8.70|9.00|
|预测值|4.72| 4.72| 4.72| 5.2 | 5.2 | 8.32| 8.32| 8.32| 8.32| 8.32


计算一下此时的预测的 mse

In [30]:
sum((y-cur_predict)**2)

2.8692000000000006

如果此时 mse 已低于要求的精度，那么就可以任务由这两棵树构成的 GBDT 已满足要求，即可停止迭代。

否则，计算此时预测的残差，作为第三棵树的拟合目标

In [32]:
y-cur_predict

array([-0.22,  0.03,  0.19,  0.14,  0.6 , -1.27, -0.42, -0.09,  0.38,
        0.68])

第三棵树的拟合目标

|x|1|2|3|4|5|6|7|8|9|10
|--|--|--|--|--|--|--|--|--|--|--|
|残差|-0.22|  0.03|  0.19|  0.14|  0.6 | -1.27| -0.42| -0.09|  0.38|0.68

继续生成第三棵树即可。